In [ ]:
%pip install aioarango
%pip install python-dotenv
%pip install pandas

In [ ]:
from aioarango import ArangoClient
import os
from dotenv import load_dotenv
from aioarango import errno, DocumentInsertError
from pandas import read_json

load_dotenv()

username_arango = os.getenv('arango_username')
password_arango = os.getenv('arango_password')
database_arango = os.getenv('arango_database')
sys_username_arango = os.getenv('sys_username_arango')
sys_password_arango = os.getenv('sys_password_arango')

print(username_arango)
print(password_arango)
print(database_arango)

client = ArangoClient(hosts="https://arangodb.roulet.dev")

# Connect to "_system" database as root user.
sys_db = await client.db("_system", username=str(sys_username_arango), password=str(sys_password_arango))

# Create a new database named "DDI_MC2".
try:
    await sys_db.create_database("DDI_MC2")
except Exception as exc:
    if (exc.error_code == errno.DUPLICATE_NAME):
        print("Database already exists")


# Connect to "test" database as root user.
db = await client.db(str(database_arango), username=str(username_arango), password=str(password_arango))

# Create a new collection named "students".
try:
    users = await db.create_collection("users")
except Exception as e:
    if (e.error_code == errno.DUPLICATE_NAME):
        print("Collection already exists users")
        users = db.collection("users")


try:
    courses = await db.create_collection("courses")
except Exception as e:
    if (e.error_code == errno.DUPLICATE_NAME):
        print("Collection already exists courses")
        courses = db.collection("courses")





# read json file to dict
import json
with open('../customers.json') as json_file:
     customer_model = json.load(json_file)


for u in customer_model:
    try:
        await users.insert(document=u, sync=False, return_new=True, silent=True, overwrite=True)
    except Exception as e:
        print(e)
        


In [ ]:
try:
    books = await db.create_collection("books")
except Exception as e:
    if (e.error_code == errno.DUPLICATE_NAME):
        print("Collection already exists books")
        books = db.collection("books")


with open('../books.json') as json_file:
     books_model = json.load(json_file)

for b in books_model:
    try:
        await books.insert(document=b, sync=True, return_new=True, silent=True, overwrite=True)
    except Exception as e:
        print(e)


In [ ]:
# Create a new graph named "school".
graph = await db.create_graph("course")

# Create vertex collections for the graph.
courses = await graph.create_vertex_collection("courses")

# Create an edge definition (relation) for the graph.
edges = await graph.create_edge_definition(
    edge_collection="register",
    from_vertex_collections=["books"],
    to_vertex_collections=["courses"]
)

# Insert vertex documents into "students" (from) vertex collection.

# Insert vertex documents into "lectures" (to) vertex collection.
await courses.insert({"_key": "GAN", "title": "Calculus"})
await courses.insert({"_key": "WER", "title": "Statistics"})
await courses.insert({"_key": "ALM", "title": "Algorithms"})
await courses.insert({"_key": "DDI", "title": "Datenbank Design"})
await courses.insert({"_key": "LLR", "title": "Lineare Regression"})
await courses.insert({"_key": "GDB", "title": "Grundlagen Datenbanken"})

# Insert edge documents into "register" edge collection.
await edges.insert({"_to": "books/5063474", "_from": "courses/DDI"})
await edges.insert({"_to": "books/5063480", "_from": "courses/DDI"})
await edges.insert({"_to": "books/5063483", "_from": "courses/DDI"})


await edges.insert({"_to": "books/5063474", "_from": "courses/GDB"})
await edges.insert({"_to": "books/5063480", "_from": "courses/GDB"})
await edges.insert({"_to": "books/5063483", "_from": "courses/GDB"})

await edges.insert({"_to": "books/5063474", "_from": "courses/LLR"})
await edges.insert({"_to": "books/5063480", "_from": "courses/LLR"})
await edges.insert({"_to": "books/5063535", "_from": "courses/LLR"})

await edges.insert({"_to": "books/5063529", "_from": "courses/ALM"})
await edges.insert({"_to": "books/5063527", "_from": "courses/ALM"})
await edges.insert({"_to": "books/5063539", "_from": "courses/ALM"})


# Traverse the graph in outbound direction, breadth-first.
result = await graph.traverse(
    start_vertex="books/5063480",
    direction="outbound",
    strategy="breadthfirst"
)

In [ ]:
# Create a new graph named "school".
graph = await db.create_graph("orders")


#Create an edge definition (relation) for the graph.
edges = await graph.create_edge_definition(
    edge_collection="order_register",
    from_vertex_collections=["users"],
    to_vertex_collections=["books"]
)

# Insert vertex documents into "students" (from) vertex collection.

# Insert edge documents into "register" edge collection.
await edges.insert({"_to": "users/3121802", "_from": "books/5063475", "order_date": "2020-01-01", "order_status": "open", "order_amount": 23.99})
await edges.insert({"_to": "users/3121802", "_from": "books/5063476", "order_date": "2020-01-01", "order_status": "open", "order_amount": 25.99})
await edges.insert({"_to": "users/3121802", "_from": "books/5063477", "order_date": "2020-01-01", "order_status": "open", "order_amount": 23.99})
await edges.insert({"_to": "users/3121802", "_from": "books/5063478", "order_date": "2020-01-01", "order_status": "open", "order_amount": 24.99})
await edges.insert({"_to": "users/3121802", "_from": "books/5063479", "order_date": "2020-01-01", "order_status": "open", "order_amount": 23.99})
await edges.insert({"_to": "users/3121802", "_from": "books/5063480", "order_date": "2020-01-01", "order_status": "open", "order_amount": 3.99})
await edges.insert({"_to": "users/3121802", "_from": "books/5063481", "order_date": "2020-01-01", "order_status": "open", "order_amount": 23.99})
await edges.insert({"_to": "users/3121802", "_from": "books/5063482", "order_date": "2020-01-01", "order_status": "open", "order_amount": 26.99})
await edges.insert({"_to": "users/3121802", "_from": "books/5063483", "order_date": "2020-01-01", "order_status": "open", "order_amount": 27.99})


await edges.insert({"_to": "users/3121808", "_from": "books/5063475", "order_date": "2020-01-03", "order_status": "open", "order_amount": 23.99})
await edges.insert({"_to": "users/3121808", "_from": "books/5063504","order_date": "2020-05-21", "order_status": "open", "order_amount": 26.99})
await edges.insert({"_to": "users/3121808", "_from": "books/5063477","order_date": "2020-02-01", "order_status": "open", "order_amount": 23.99})
await edges.insert({"_to": "users/3121808", "_from": "books/5063478","order_date": "2020-01-01", "order_status": "open", "order_amount": 25.99})
await edges.insert({"_to": "users/3121808", "_from": "books/5063479","order_date": "2020-01-01", "order_status": "open", "order_amount": 23.99})
await edges.insert({"_to": "users/3121808", "_from": "books/5063506","order_date": "2020-01-01", "order_status": "open", "order_amount": 26.99})
await edges.insert({"_to": "users/3121808", "_from": "books/5063481","order_date": "2020-04-31", "order_status": "open", "order_amount": 23.99})
await edges.insert({"_to": "users/3121808", "_from": "books/5063507","order_date": "2020-01-01", "order_status": "open", "order_amount": 277.99})
await edges.insert({"_to": "users/3121808", "_from": "books/5063483","order_date": "2020-03-01", "order_status": "open", "order_amount": 28.99})


await edges.insert({"_to": "users/3121837", "_from": "books/5063475","order_date": "2020-02-05", "order_status": "open", "order_amount": 23.99})
await edges.insert({"_to": "users/3121837", "_from": "books/5063476","order_date": "2020-03-01", "order_status": "open", "order_amount": 25.99})
await edges.insert({"_to": "users/3121837", "_from": "books/5063477","order_date": "2020-01-02", "order_status": "open", "order_amount": 23.99})
await edges.insert({"_to": "users/3121837", "_from": "books/5063501","order_date": "2020-04-01", "order_status": "open", "order_amount": 2.99})
await edges.insert({"_to": "users/3121837", "_from": "books/5063479","order_date": "2020-01-03", "order_status": "open", "order_amount": 23.99})
await edges.insert({"_to": "users/3121837", "_from": "books/5063504","order_date": "2020-01-01", "order_status": "open", "order_amount": 25.99})
await edges.insert({"_to": "users/3121837", "_from": "books/5063481","order_date": "2020-05-01", "order_status": "open", "order_amount": 23.99})
await edges.insert({"_to": "users/3121837", "_from": "books/5063503","order_date": "2020-01-01", "order_status": "open", "order_amount": 3.99})
await edges.insert({"_to": "users/3121837", "_from": "books/5063483","order_date": "2020-02-01", "order_status": "open", "order_amount": 23.99})

# Traverse the graph in outbound direction, breadth-first.
result = await graph.traverse(
    start_vertex="users/3121802",
    direction="outbound",
    strategy="breadthfirst"
)

In [ ]:
#cursor = await db.aql.execute("FOR u IN users RETURN u")
#users = [document["name"] async for document in cursor]
#print(users)